In [ ]:
import preprocess as pp
import configs
import persister
import relation
import lda

import re
import json

from stanfordcorenlp import StanfordCoreNLP
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.tree import Tree
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
# 文档向量化测试
# count vectorizer

In [ ]:
a = "Many existing knowledge bases(KBs), including Freebase, Yago, and NELL, rely on a ﬁxed ontology, given as an input to the system, which deﬁnes the data to be cataloged in the KB, i.e., a hierarchy of categories and relations between them. The system then extracts facts that match the predeﬁned ontology. We propose an unsupervised model that jointly learns a latent ontological structure of an input corpus, and identiﬁes facts from the corpus that match the learned structure. Our approach combines mixed membership stochastic block models and topic models to infer a structure by jointly modeling text, a latent concept hierarchy, and latent semantic relationships among the entities mentioned in the text. As a case study, we apply the model to a corpus of Web documents from the software domain,and evaluate the accuracy of the various components of the learned ontology. "
b = ["hello world. what the fuck."]

In [ ]:
c = [" ".join(preprocess(a))]
vector = CountVectorizer(ngram_range=(1, 2), vocabulary=["knowledge base"], stop_words='english')
vector.build_analyzer()
x = vector.fit_transform(c)

In [ ]:
estimator = CountVectorizer(ngram_range=(1, 2), vocabulary=['fuck', 'hello', 'world', "hello world"], stop_words='english')
estimator.build_analyzer()
res = estimator.fit_transform(b)

In [ ]:
estimator.get_feature_names()

In [ ]:
res.toarray()
# estimator.vocabulary_

In [ ]:
# 斯坦福nlp工具测试
# stanfordcorenlp

In [ ]:
nlp = StanfordCoreNLP('/mnt/d/stanford-corenlp-full-2018-02-27', port=9000)
sentence = 'Brink is taking part in the final exercise.'
print('Tokenize:', nlp.word_tokenize(sentence))
print('Part of Speech:', nlp.pos_tag(sentence))
print('Named Entities:', nlp.ner(sentence))
print('Constituency Parsing:', nlp.parse(sentence))
print('Dependency Parsing:', nlp.dependency_parse(sentence))

nlp.close() # Do not forget to close! The backend server will consume a lot memery.

In [ ]:
# 官方corenlp api
# stanza

In [ ]:
from stanza.server import CoreNLPClient
with CoreNLPClient(properties="./corenlp_server.props", timeout=30000, memory='5G') as client:
    res = relation.corenlp_annotate(client, pp.format_news(newsdata[0]))

In [ ]:
# 新闻数据集测试
# 20newsgroups

In [ ]:
import newsgroups
size = 500
tmp = newsgroups.get_news_data(size)
newsdata = []
for cate in tmp:
    newsdata.extend(cate)

In [ ]:
texts = [' '.join(pp.preprocess_abstract(a))
persister.save_json(configs.NEWSDATA, texts)

In [ ]:
# 处理成句子
# nltk.sent_tokenize

In [ ]:
newssent = []
for news in newsdata:
    newssent.append(pp.split2sent(news))
persister.save_json(configs.NEWSSENT, newssent)

In [ ]:
newssenttoken = []
for news in newssent:
    tokenized_lemmatized_news = []
    for sent in news:
        tokenized_lemmatized_news.append(relation.lemmatize_sent_words(sent))
    newssenttoken.append(tokenized_lemmatized_news)
persister.save_json(configs.NEWSSENTTOKEN, newssenttoken)

In [ ]:
# load preprossed data

In [ ]:
rawnews = persister.load_json(configs.RAWNEWS)
newsparse = persister.read_parse()

In [ ]:
# load lda res
news_input = persister.read_input(configs.NEWSINPUT)
terms, doc_topic, topic_word = persister.read_lda(configs.NEWSLDA)
top_terms, top_docs = lda.get_topics(topic_word, terms, doc_topic)

In [ ]:
# use gensim compute coherence
texts = [word_tokenize(corp) for corp in news_input]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
m = CoherenceModel(topics=[[t[0] for t in topic] for topic in top_terms], 
                   corpus=corpus, dictionary=dictionary, coherence='u_mass')
m.get_coherence_per_topic()

In [ ]:
# use tmtool select model by coherence
tf, vector_model = lda.extract_feature(news_input)
m, c_lst = lda.select_model_by_coherence(tf, np.array(vector_model.get_feature_names()), news_input, "c_v", range(2,8,2), max_iter=100)

In [ ]:
# column names
topicnames = ["Topic" + str(i) for i in range(len(topic_word))]

# index names
docnames = ["Doc" + str(i) for i in range(len(doc_topic))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(doc_topic, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic[10:20].style.applymap(color_green).applymap(make_bold)
df_document_topics

In [ ]:
from collections import Counter
Counter(lda.get_dominant_topic(doc_topic, topic_word))

In [ ]:
top_terms_vis, top_docs_vis = lda.pd_topics_vis(top_terms,top_docs)
top_terms_vis

In [ ]:
param = lda.generate_lda_parameter(2,8,2,[20])
tf, _ = lda.extract_feature(news_input)
model = lda.gridsearchCV(param, tf)

In [ ]:
print(model.best_estimator_.perplexity(tf))
pd.DataFrame(model.cv_results_)

In [ ]:
n_topics = list(range(2,8,2))
score = 'mean_test_score'
log_likelyhoods_5 = [round(s) for s in model.cv_results_[score][:len(n_topics)]] 
log_likelyhoods_7 = [round(s) for s in model.cv_results_[score][len(n_topics):2*len(n_topics)]]
log_likelyhoods_9 = [round(s) for s in model.cv_results_[score][2*len(n_topics):3*len(n_topics)]]

# Show graph
plt.figure(figsize=(12, 8))
print(log_likelyhoods_5)
print(log_likelyhoods_7)
plt.plot(n_topics, log_likelyhoods_5, label='0.5')
plt.plot(n_topics, log_likelyhoods_7, label='0.7')
plt.plot(n_topics, log_likelyhoods_9, label='0.9')
plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.show()

In [ ]:
tf, _ = lda.extract_feature(news_input)
m,perps = lda.select_model_by_perplexity(tf, range(2,8,1), learning_decay=0.8, max_iter=30)
perps

In [ ]:
from functools import reduce
failed_idxs = []
for idx, i in enumerate(newsparse):
    if type(i) == str:
        failed_idxs.append(idx)
print(len(set(failed_idxs) & set(map(lambda x:x[0], reduce(lambda x,y:x+y, top_docs)))))
print(len(set(map(lambda x:x[0], reduce(lambda x,y:x+y, top_docs)))))

In [ ]:
idx_news = 1195
idx_sent = 40

In [ ]:
print(pos_tag(word_tokenize(newssent[idx_news][43])))
print(pp.WNL.lemmatize("has", wordnet.VERB))

In [ ]:
print(len(newsparse[idx_news]["sentences"]))
print(newsparse[idx_news]["sentences"][idx_sent].keys())
examparse = newsparse[idx_news]["sentences"][idx_sent]
print(examparse["enhancedPlusPlusDependencies"])

In [ ]:
tt = Tree.fromstring(examparse["parse"])
tt.pretty_print()

In [ ]:
t4 = [t[0] for t in top_terms[3]]
t4_values = [t[1] for t in top_terms[3]]
print(t4)
print(t4_values)
idxs, importance, count = relation.extract_important_sents(newssenttoken[idx_news], t4, t4_values)
print(idxs, importance[40], count[40])

In [ ]:
relation.extract_word_relation_from_sent(i, newsparse[1200]["sentences"][sent_idx]["enhancedDependencies"])

In [ ]:
for term in top_terms[3]:
    for news in top_docs[3]:
        time.sleep(2)
        news_idx = news[0]
        for sent_idx,sent in enumerate(newssenttoken[news_idx]):
            if type(newsparse[news_idx]) is str:
                print(news_idx, "parse", "err")
                break
            topic_word_idxs = relation.get_word_idx(term[0], sent)
            for i in topic_word_idxs:
                print(sent_idx, i, news_idx)
                rs = relation.extract_word_relation_from_sent(i, newsparse[news_idx]["sentences"][sent_idx]["enhancedDependencies"])
                for r in rs:
                    print(relation.convert_relation2str(r))

In [ ]:
t = Tree.fromstring(newsparse[idx_news]["sentences"][5]["parse"])
t.pretty_print()

In [ ]:
# abstract data

In [ ]:
raw_data = persister.load_json(configs.RAWABSTRACT)
abslda_input = persister.read_input(configs.ABSTRACTINPUT)
terms, doc_topic, topic_word = persister.read_lda(configs.ABSTRACTLDA)
absparse = persister.read_parse(configs.ABSTRACTPARSE)

In [ ]:
param = lda.generate_lda_parameter(5,30,5,10)
vec, _ = lda.extract_feature(abslda_input)
model = lda.gridsearchCV(param, vec)

In [ ]:
n_topics = list(range(5,30,5))
score = 'mean_test_score'
# score = 'std_test_score'
log_likelyhoods_5 = [round(s) for s in model.cv_results_[score][:len(n_topics)]] 
log_likelyhoods_7 = [round(s) for s in model.cv_results_[score][len(n_topics):2*len(n_topics)]]
log_likelyhoods_9 = [round(s) for s in model.cv_results_[score][2*len(n_topics):3*len(n_topics)]]

# Show graph
plt.figure(figsize=(12, 8))
print(log_likelyhoods_5)
print(log_likelyhoods_7)
plt.plot(n_topics, log_likelyhoods_5, label='0.5')
plt.plot(n_topics, log_likelyhoods_7, label='0.7')
plt.plot(n_topics, log_likelyhoods_9, label='0.9')
plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.show()

In [ ]:
import pyLDAvis
pyLDAvis.enable_notebook(True)
panel = pyLDAvis.prepare(topic_word, doc_topic, [len(s) for s in [word_tokenize(corp) for corp in news_input]], vector_model.get_feature_names(), np.array(sum(tf).todense())[0])
pyLDAvis.show(panel)